In [ ]:
# https://openrouter.ai/settings/keys

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.0 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.4 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 40.8 MB/s eta 0:00:00


In [5]:
!git clone https://github.com/Jaish19/GenAI---RAG-using-LangChain.git

Cloning into 'GenAI---RAG-using-LangChain'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 93 (delta 42), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 8.29 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [7]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from google.colab import userdata

# Step 1: Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = userdata.get("OPENROUTER_OPENAPI") # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=True
)

# Step 3: Interact with the model (normal chat mode)
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = llm([HumanMessage(content=query)])
    print("\nAssistant:", response.content)


/tmp/ipython-input-202412850.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(



You: hi


/tmp/ipython-input-202412850.py:27: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=query)])



Assistant:  Hello! How can I help you today? If you have any questions or need assistance with something, feel free to ask. I'm here to help you.

You: pawan kalyan

Assistant:  Pawan Kalyan is a popular Indian film actor, producer, and politician, predominantly known for his works in the Telugu film industry. He is the younger brother of the famous actor Chiranjeevi and is a Member of Parliament in the Rajya Sabha, representing the state of Andhra Pradesh.

Pawan Kalyan made his acting debut in 1998 with the film "AK-47" and has since appeared in several successful films such as "Gabbar Singh," "Attarintiki Daredi," and "Sardaar Gabbar Singh." He is known for his action roles and has also established himself as a versatile actor with films like "Jalsa" and "Johnny."

In addition to his acting career, Pawan Kalyan also co-founded the political party Jana Sena Party in 2014, which aims to address the issues of farmers, the middle class, and the youth. He served as a member of the Telan

KeyboardInterrupt: Interrupted by user

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA # RAG - To retrieve the contents from the pdfs or any docs
from langchain.chains.llm import LLMChain  # It's LangChain function to wrap the tools/agents to create a seemless workflow
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate  # To provide the equipped prompt
from langchain_community.document_loaders import PDFPlumberLoader # To read the pdf
from langchain_experimental.text_splitter import SemanticChunker # Text chunks
from langchain_community.embeddings import HuggingFaceEmbeddings # Embed the texts or contents
from langchain_community.vectorstores import FAISS  # Vector DB
from google.colab import userdata

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = userdata.get("OPENROUTER_OPENAPI")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"  # Endpoint of the openRouter

# Load and process PDF
loader = PDFPlumberLoader("/content/GenAI---RAG-using-LangChain/neural_network.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))


# PROCESS 1: CHUNK process
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Semantic splitter with custom
# With SemanticChunker (in LangChain), you don't define chunk_size directly like in other splitters
# (CharacterTextSplitter, RecursiveCharacterTextSplitter, etc.).
# Instead, semantic chunking decides where to split based on semantic similarity between segments, not on fixed sizes.
splitter = SemanticChunker(
    embeddings=HuggingFaceEmbeddings(),
    breakpoint_threshold_type="percentile", # or standard_deviation
    breakpoint_threshold_amount=90, # Value for the threshold (e.g. 95 = 95th percentile distance between embeddings)
    buffer_size=1 # (Optional) Adds context from neighboring chunks
)

# Split document
chunks = splitter.split_documents(docs)
for chunk in chunks:
    print(chunk.page_content)


# PROCESS 2: Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(chunks, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

'''
# Try with some queries for similarity search
results = retriever.get_relevant_documents("What is the remedy for cold and flu?")

for doc in results:
    print(doc.page_content)
'''

# PROCESS 3: LLM MODEL CHOOSING
# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3-8b-instruct",   #"mistralai/mistral-7b-instruct"
    temperature=1.3, # It's a less temp -> so it'll give you some relative information about the content.
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# PROCESS 4: PROMPT CREATION
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)


# PROCESS 5: LANGCHAIN CREATION - LLM & PROMPT
llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

# DOCUMENT CHAIN PROCESS
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

# PROCESS 6: WRAPPING ALL TOGETHER ALONG WITH RETRIEVER
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is the remedy for cold and flu?")
print("Answer:", result["result"])


Pages loaded: 224


/tmp/ipython-input-1164428662.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(),
/tmp/ipython-input-1164428662.py:31: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings(),
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hugg

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Example query
result = qa("What is the cost function?")
print("Answer:", result["result"])